In [ ]:
import numpy as np, pandas as pd, cvxpy as cp, cvxopt
from collections import Counter
from sklearn.metrics import confusion_matrix

import cvxpy as cp

## 1' import data 

In [ ]:
train = pd.concat([pd.read_csv("X_labeled.csv"), pd.read_csv("y_labeled.csv")],1,names =None)
train = train.drop(train.columns[[0,56]],1)
aux = pd.read_csv("X_unlabeled.csv")
aux = aux.drop(aux.columns[0],1)

print(Counter(train["target"]))
print(aux.shape)
print(train[train["school"]=="GP"].shape)
print(train[train["school"]=='MS'].shape)
print(aux[aux["school"]=="GP"].shape)
print(aux[aux["school"]=="MS"].shape)

## 2' Define Function S3VMplus

In [ ]:
def S3VMplus(train,aux,C_pos,C_neg,gamma):
    
    F = train.shape[1]
        
    Xt1_pos = np.matrix(train[(train["target"]==1) & (train["school"]=="GP")].iloc[:,1:-1])
    Yt1_pos = np.array(train[(train["target"]==1) & (train["school"]=="GP")].iloc[:,-1])
    
    Xt1_neg = np.matrix(train[(train["target"]==-1) & (train["school"]=="GP")].iloc[:,1:-1])
    Yt1_neg = np.array(train[(train["target"]==-1) & (train["school"]=="GP")].iloc[:,-1])
    
    Xt2_pos = np.matrix(train[(train["target"]==1) & (train["school"]=="MS")].iloc[:,1:-1])
    Yt2_pos = np.array(train[(train["target"]==1) & (train["school"]=="MS")].iloc[:,-1])
    
    Xt2_neg = np.matrix(train[(train["target"]==-1) & (train["school"]=="MS")].iloc[:,1:-1])
    Yt2_neg = np.array(train[(train["target"]==-1) & (train["school"]=="MS")].iloc[:,-1])
    
    aux1 = np.matrix(aux[aux["school"]=="GP"].iloc[:,1:])
    aux2 = np.matrix(aux[aux["school"]=="MS"].iloc[:,1:])
    
    N1_pos,F = Xt1_pos.shape
    N1_neg,F = Xt1_neg.shape
    
    N2_pos,F = Xt2_pos.shape
    N2_neg,F = Xt2_neg.shape
    
    K1 = aux1.shape[0]
    K2 = aux2.shape[0]
    
    C1_pos = C_pos*np.ones(N1_pos)
    C1_neg = C_neg*np.ones(N1_neg)
    C2_pos = C_pos*np.ones(N2_pos)
    C2_neg = C_pos*np.ones(N2_neg)
    C1_aux_pos = C_pos*np.ones(K1)
    C1_aux_neg = C_neg*np.ones(K1)
    C2_aux_pos = C_pos*np.ones(K2)
    C2_aux_neg = C_neg*np.ones(K2)
    
    M = 1000000
    
    w = cp.Variable(F)
    b = cp.Variable()
    w1 = cp.Variable(F)
    d1 = cp.Variable()
    w2 = cp.Variable(F)
    d2 = cp.Variable()
    
    #below is slacks
    e1_pos = cp.Variable(N1_pos)
    e2_pos = cp.Variable(N2_pos)
    e1_neg = cp.Variable(N1_neg)
    e2_neg = cp.Variable(N2_neg)
    
    ##aux
    xi1 = cp.Variable(K1)
    xi2 = cp.Variable(K2)
    z1 = cp.Variable(K1)
    z2 = cp.Variable(K2)
    
    #binary for aux
    d1_bi = np.ones(K1)
    d2_bi = np.ones(K2)
    
    objective = cp.Minimize(0.5*cp.square(cp.norm(w))+
                            0.5*gamma*(cp.square(cp.norm(w1))+
                                       cp.square(cp.norm(w2))
                                      )+
                            cp.sum(cp.multiply(C1_pos,e1_pos))+
                            cp.sum(cp.multiply(C1_neg,e1_neg))+
                            cp.sum(cp.multiply(C2_pos,e2_pos))+
                            cp.sum(cp.multiply(C2_neg,e2_neg))+
                            cp.sum(cp.multiply(C1_aux_pos,xi1))+
                            cp.sum(cp.multiply(C2_aux_pos,xi2))+
                            cp.sum(cp.multiply(C1_aux_neg,z1))+
                            cp.sum(cp.multiply(C2_aux_neg,z2))
                           )
                            
    constraints=[cp.multiply(Yt1_pos.T,((Xt1_pos*w+b)+(Xt1_pos*w1+d1)))+e1_pos-1 >= 0,
                 e1_pos>=0,
                 cp.multiply(Yt1_neg.T,((Xt1_neg*w+b)+(Xt1_neg*w1+d1)))+e1_neg-1 >= 0,
                 e1_neg>=0,
                 cp.multiply(Yt2_pos.T,((Xt2_pos*w+b)+(Xt2_pos*w2+d2)))+e2_pos-1 >= 0,
                 e2_pos>=0,
                 cp.multiply(Yt2_neg.T,((Xt2_neg*w+b)+(Xt2_neg*w2+d2)))+e2_neg-1 >= 0,
                 e2_neg>=0,
                 ((aux1*w+b)+(aux1*w1+d1))+xi1+M*(1-d1_bi)-1>=0,
                 ((aux1*w+b)+(aux1*w1+d1))+xi1+M*d1_bi-1>=0,
                 xi1>=0,
                 ((aux2*w+b)+(aux2*w2+d2))+xi2+M*(1-d2_bi)-1>=0,
                 ((aux2*w+b)+(aux2*w2+d2))+xi2+M*d2_bi-1>=0,
                 xi2>=0,
                 -((aux1*w+b)+(aux1*w1+d1))+z1+M*d1_bi-1>=0,
                 -((aux1*w+b)+(aux1*w1+d1))+z1+M*(1-d1_bi)-1>=0,
                 z1>=0,
                 -((aux2*w+b)+(aux2*w2+d2))+z2+M*d2_bi-1>=0,
                 -((aux2*w+b)+(aux2*w2+d2))+z2+M*(1-d2_bi)-1>=0,
                 z2>=0]
    
    prob = cp.Problem(objective,constraints)
    
    try:
        results = prob.solve()
    except:
        try:
            results = prob.solve(solver = "ECOS_BB")
        except:
            results = prob.solve(solver = "SCS")
                            
    return w.value,b.value,w1.value,w2.value,d1.value,d2.value

## 3' Define Training Accuracy Function

In [ ]:
def train_accuracy(w,b,w1,d1,w2,d2,train):
    countin = 0
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP":
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1)*train.iloc[i][-1] >0:
                countin += 1
        elif train.iloc[i][0] == "MS":
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2)*train.iloc[i][-1] >0:
                countin += 1
    return countin/train.shape[0]

## 4' Select Parameters (C_pos,C_neg,gamma)

In [ ]:
def cmgc(C_1,C_2,g):
    
    train_result = []
    w,b,w1,w2,d1,d2 = S3VMplus(train,aux,C_1,C_2,g)
    
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP" :
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1) > 0 :
                train_result.append(1)
            else:
                train_result.append(-1)
        else:
            if (sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2) > 0 :
                train_result.append(1)
            else:
                train_result.append(-1)

    return confusion_matrix(train["target"],train_result)

gamma_list = (0.1, 0.2, 0.5, 0.8,1, 2, 5, 10, 20, 30, 40, 50,100)

diff_thre = 0.06
lower_thre = 1.65
#upper_thre = 
info = []
candidates = []
for gamma in gamma_list:
    for C_p in 2**np.linspace(-8,8,17):
        for C_n in 2**np.linspace(-8,8,17):
            temp = cmgc(C_p,C_n,gamma)
            acc_sum = temp[0,0]/sum(temp[0,:]) + temp[1,1]/sum(temp[1,:])
            acc_diff = abs(temp[0,0]/sum(temp[0,:]) - temp[1,1]/sum(temp[1,:]))
            info.append([C_p,C_n,gamma,acc_sum,acc_diff])
            if acc_sum >lower_thre and acc_diff<diff_thre:
                print("pos:",C_p,"neg:",C_n,"gamma:",gamma,"acc_sum:",acc_sum,"acc_diff:",acc_diff)
                candidates.append([C_p,C_n,gamma,acc_sum,acc_diff])
pd.DataFrame(info).to_csv("info_S3VMplus.csv")

In [ ]:
#load in after save
info = pd.DataFrame.from_csv("info_S3VMplus.csv")
info.iloc[:,3] = np.round(info.iloc[:,3],decimals = 2)
info.iloc[:,4] = np.round(info.iloc[:,4],decimals = 2)
reduced_info = info.drop_duplicates(["3","4"])
selected = reduced_info.loc[(reduced_info.iloc[:,3]>max(info.iloc[:,3])-0.1)&(reduced_info.iloc[:,4]<0.3),:]
print(selected.shape)
print(selected)

In [ ]:
#table = pd.DataFrame(info)
#candidates = table.loc[(table.iloc[:,3]>1.75)&(table.iloc[:,4]<0.05),:]

## 5' Training Accuracy and Confusion  Matrix

In [ ]:
result_combo = []

for j in np.arange(0,len(selected)):
    
    C_pos,C_neg,gamma = selected.iloc[j,0:3]
    w,b,w1,w2,d1,d2 = S3VMplus(train,aux,C_pos,C_neg,gamma)
    count = 0
    result = []
    
    for i in range(0,train.shape[0]):
        if train.iloc[i][0] == "GP" :
            result.append(sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w1)+d1)
        else:
            result.append(sum(train.iloc[i][1:-1]*w)+b+sum(train.iloc[i][1:-1]*w2)+d2)
    
    result = np.array(result)/abs(np.array(result))
    result_combo.append(result)
    print(j+1,":training confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")

voted = np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo)/abs(np.matrix(result_combo))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted))
print("%f" % ((confusion_matrix(train["target"],voted)[0,0]+confusion_matrix(train["target"],voted)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted)[0,0]/sum(confusion_matrix(train["target"],voted)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted)[1,1]/sum(confusion_matrix(train["target"],voted)[1,:])),"%")

## Test Accuracy and Confusion Matrix

In [ ]:
result_combo_test = []

for j in np.arange(0,len(selected)):
    
    C_pos,C_neg,gamma = selected.iloc[j,0:3]
    result = []
    
    for i in range(0,train.shape[0]):
        X_test = train.iloc[i,0:-1]
        w,b,w1,w2,d1,d2 = S3VMplus(train.drop(train.index[i]),aux,C_pos,C_neg,gamma)
        if X_test[0] == "GP" :
            result.append(sum(X_test[1:]*w)+b+sum(X_test[1:]*w1)+d1)
        elif X_test[0] == "MS":
            result.append(sum(X_test[1:]*w)+b+sum(X_test[1:]*w2)+d2)
    
    result = np.array(result)/abs(np.array(result))
    result_combo_test.append(result)
    print(j+1,":test confusion matrix:\n",confusion_matrix(train["target"],result))
    print("%f" % ((confusion_matrix(train["target"],result)[0,0]+confusion_matrix(train["target"],result)[1,1])/train.shape[0]),"%")

voted_test = np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test)))))/abs(np.squeeze(np.asarray(sum(np.matrix(result_combo_test)/abs(np.matrix(result_combo_test))))))
print("voted confusion matrix:\n",confusion_matrix(train["target"],voted_test))
print("%f" % ((confusion_matrix(train["target"],voted_test)[0,0]+confusion_matrix(train["target"],voted_test)[1,1])/train.shape[0]),"%")
print("-1:","%f" % (confusion_matrix(train["target"],voted_test)[0,0]/sum(confusion_matrix(train["target"],voted_test)[0,:])),"%")
print("1:","%f" % (confusion_matrix(train["target"],voted_test)[1,1]/sum(confusion_matrix(train["target"],voted_test)[1,:])),"%")